<a href="https://colab.research.google.com/github/amolaka/DS-3001---Voting-Project/blob/main/combining_model_%26_voting_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import necessary packages
! pip3 install mapclassify
import geopandas as gpd
import mapclassify
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
# Load data
md = pd.read_csv('./model_data.csv') # Load in model_data (county data)
vdf = pd.read_csv('./voting_VA_clean.csv') # Load historical voting data


In [4]:
vdf.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,candidatevotes_log,totalvotes_log
0,0,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL,8.535426,9.386392
1,1,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL,8.756525,9.386392
2,2,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL,5.393628,9.386392
3,3,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL,5.564520,9.386392
4,4,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL,9.696156,10.514502


In [5]:
md.head()

,Unnamed: 0,County Name,County Code,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,0-50K,50K-100K,...,Male: Some early_log,Male: High school_log,Male: Some college or Associate's degree_log,Male: Bachelor's degree_log,Male: Master's degree or higher_log,Female: Some early_log,Female: High school_log,Female: Some college or Associate's degree_log,Female: Bachelor's degree_log,Female: Master's degree or higher_log
0,0,Accomack County,1.0,394.0,84.0,32.0,44.0,127.0,1702.0,666.0,...,6.086775,6.814543,6.274762,5.411646,5.786897,5.762051,6.833032,6.419995,5.537334,5.710427
1,1,Albemarle County,3.0,509.0,328.0,140.0,223.0,43.0,2158.0,1278.0,...,6.523562,6.971669,6.626718,6.342121,6.838405,6.313548,6.967909,6.938284,6.395262,6.837333
2,2,Alleghany County,5.0,75.0,113.0,5.0,70.0,127.0,1012.0,447.0,...,5.579730,6.148468,5.846439,4.890349,4.779123,5.293305,6.253829,5.942799,5.036953,5.627621
3,3,Amelia County,7.0,152.0,212.0,38.0,53.0,28.0,978.0,515.0,...,5.796058,6.326149,5.877736,4.976734,5.187386,4.997212,6.364751,6.049733,4.927254,5.863631
4,4,Amherst County,9.0,323.0,162.0,85.0,51.0,127.0,1406.0,635.0,...,6.016157,6.699500,6.304449,5.117994,5.370638,6.284134,6.689599,6.298949,5.365976,5.583496


## joining the two dfs on county name

In [6]:

# Step 1: Normalize county names
vdf['county_name'] = vdf['county_name'].str.lower()
md['County Name'] = md['County Name'].str.lower()

# Step 2: Remove "County" from one data frame
md['County Name'] = md['County Name'].str.replace('county', '')

# Step 3: Rename columns
vdf.rename(columns={'county_name': 'county'}, inplace=True)  # Replace old column names with just 'county'
md.rename(columns={'County Name': 'county'}, inplace=True)

# Step 4: Perform the join operation
merged_df = pd.merge(vdf, md, on='county')

# Optionally, you can revert the county names to their original capitalization
merged_df['county'] = merged_df['county'].str.title()

In [7]:
merged_df.head()

,Unnamed: 0_x,year,state,state_po,county,county_fips,office,candidate,party,candidatevotes,...,Male: Some early_log,Male: High school_log,Male: Some college or Associate's degree_log,Male: Bachelor's degree_log,Male: Master's degree or higher_log,Female: Some early_log,Female: High school_log,Female: Some college or Associate's degree_log,Female: Bachelor's degree_log,Female: Master's degree or higher_log
0,3284,2020,VIRGINIA,VA,Alexandria City,51510,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,55940,...,6.822197,7.257003,7.013915,6.666957,7.125283,6.583409,7.272398,7.104965,6.582025,7.053586
1,3284,2020,VIRGINIA,VA,Alexandria City,51510,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,55940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3284,2020,VIRGINIA,VA,Alexandria City,51510,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,55940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3284,2020,VIRGINIA,VA,Alexandria City,51510,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,55940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3284,2020,VIRGINIA,VA,Alexandria City,51510,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,55940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
merged_df.to_csv("combined_data.csv")


In [15]:
merged_df.shape

Number of rows: 3616
Number of columns: 69


In [16]:
vdf.shape

(3736, 15)

In [17]:
md.shape

(1067, 55)